In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression

# Импортирование и просмотр данных

In [2]:
forecast = pd.read_excel('Forecast_of_discounts.xlsx')
mapping = pd.read_csv('Mapping.csv')
shipments = pd.read_csv('Shipments_by_PO.csv')

In [3]:
shipments.head()

,Код товара,Канал,Дата накладной,"Продажи, шт",скидка
0,299,Канал 1,06.01.2017,32,19
1,299,Канал 1,09.01.2017,59,19
2,299,Канал 1,10.01.2017,22,2
3,299,Канал 1,11.01.2017,35,10
4,299,Канал 1,12.01.2017,55,4


In [4]:
forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1105 entries, 0 to 1104
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Код товара          1105 non-null   object 
 1   Направление продаж  1105 non-null   object 
 2   Month               1105 non-null   int64  
 3   Year                1105 non-null   int64  
 4   Скидка              1105 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 43.3+ KB


In [5]:
mapping

,Канал,Направление продаж
0,Канал 1,Розничная торговля
1,Канал 2,Розничная торговля
2,Канал 3,Розничная торговля
3,Канал 4,Розничная торговля
4,Канал 5,Розничная торговля
5,Канал 6,Оптовая торговля
6,Канал 7,Оптовая торговля
7,Канал 8,Оптовая торговля
8,Канал 9,Оптовая торговля
9,Канал 10,Оптовая торговля


In [6]:
forecast.tail()

,Код товара,Направление продаж,Month,Year,Скидка
1100,SKU_0654,Розничная торговля,7,2018,15.000000
1101,SKU_0654,Розничная торговля,9,2018,4.500000
1102,SKU_0656,Розничная торговля,8,2018,12.000000
1103,SKU_0657,Оптовая торговля,7,2018,14.428571
1104,SKU_0657,Розничная торговля,7,2018,7.000000


In [7]:
list_code_init = shipments['Код товара'].tolist()

# Создание тренировочного датасета

Будем опираться на нашу целевую таблицу с прогнозом скидок, которую будем использовать для нецелевых признаков финальной таблицы с решением - **forecast**. Для того, чтобы привести нашу изначальную таблицу **shipments** к ее виду нам нужно: сменить столбец "канал" на "направление продаж" через таблицу **mapping**, далее разделим столбец "дата накладной" на месяц и год, поскольку информация по поводу того, как определить скидку для целого месяца не дана, определим ее через медиану, а продажи по каждому месяцу для каждого канала и товара сложим. Также необходимо преобразовать "Код товара" из формата "SKU_xyz" в числовой.

Поскольку мы будем группировать по дате, сразу опустим день

In [8]:
shipments['Дата накладной']= shipments['Дата накладной'].apply(lambda x: x[3:])

Сразу сгруппируем по коду, каналу и дате. Возьмем медиану по скидке и просуммируем по месяцу пнаши продажи, поскольку в таблице **forecast** данные даны также по месяцам.

In [9]:
init_df = pd.DataFrame()
for code, sub_df in shipments.groupby(['Код товара','Канал','Дата накладной']):
    sub_df['Продажи, шт'] = sub_df['Продажи, шт'].agg(sum)
    sub_df['скидка'] = sub_df['скидка'].median()
    sub_df.drop_duplicates(inplace=True)
    init_df = pd.concat([init_df, sub_df])
    
init_df

,Код товара,Канал,Дата накладной,"Продажи, шт",скидка
189775,1,Канал 1,01.2017,1052,10.0
189999,1,Канал 1,01.2018,958,6.5
189787,1,Канал 1,02.2017,1241,8.0
190010,1,Канал 1,02.2018,868,13.0
189802,1,Канал 1,03.2017,1630,11.0
...,...,...,...,...,...
507483,657,Канал 9,06.2018,307,13.0
507410,657,Канал 9,07.2017,52,2.0
507411,657,Канал 9,10.2017,517,9.0
507423,657,Канал 9,11.2017,785,11.5


In [10]:
training_df = pd.concat([init_df], ignore_index = True)

In [11]:
training_df['Month'] = training_df['Дата накладной'].apply(lambda x: int(x[:2]))
training_df['Year'] = training_df['Дата накладной'].apply(lambda x: x[3:])
training_df.drop(columns = ['Дата накладной'], axis = 1, inplace = True)
training_df

,Код товара,Канал,"Продажи, шт",скидка,Month,Year
0,1,Канал 1,1052,10.0,1,2017
1,1,Канал 1,958,6.5,1,2018
2,1,Канал 1,1241,8.0,2,2017
3,1,Канал 1,868,13.0,2,2018
4,1,Канал 1,1630,11.0,3,2017
...,...,...,...,...,...,...
86520,657,Канал 9,307,13.0,6,2018
86521,657,Канал 9,52,2.0,7,2017
86522,657,Канал 9,517,9.0,10,2017
86523,657,Канал 9,785,11.5,11,2017


Поскольку в задании сказано, что предсказать значения нужно в для товара-направления, заменим столбец "Канал" в таблице

In [12]:
dic_map = dict(mapping.values)
training_df['Канал'] =training_df['Канал'].map(dic_map)

In [13]:
training_df = training_df.rename(columns = {'Канал': 'Направление продаж'})

In [14]:

training_df = training_df[['Код товара', 'Направление продаж', 'Month','Year','скидка','Продажи, шт']]
training_df.rename(columns = {'скидка': 'Скидка'})
training_df

,Код товара,Направление продаж,Month,Year,скидка,"Продажи, шт"
0,1,Розничная торговля,1,2017,10.0,1052
1,1,Розничная торговля,1,2018,6.5,958
2,1,Розничная торговля,2,2017,8.0,1241
3,1,Розничная торговля,2,2018,13.0,868
4,1,Розничная торговля,3,2017,11.0,1630
...,...,...,...,...,...,...
86520,657,Оптовая торговля,6,2018,13.0,307
86521,657,Оптовая торговля,7,2017,2.0,52
86522,657,Оптовая торговля,10,2017,9.0,517
86523,657,Оптовая торговля,11,2017,11.5,785


In [15]:
forecast['Код товара'] = forecast['Код товара'].apply(lambda x: int(x[4:]))
forecast

,Код товара,Направление продаж,Month,Year,Скидка
0,1,Розничная торговля,7,2018,10.000000
1,2,Оптовая торговля,7,2018,7.400000
2,4,Розничная торговля,8,2018,15.500000
3,5,Оптовая торговля,9,2018,10.857143
4,6,Оптовая торговля,7,2018,12.500000
...,...,...,...,...,...
1100,654,Розничная торговля,7,2018,15.000000
1101,654,Розничная торговля,9,2018,4.500000
1102,656,Розничная торговля,8,2018,12.000000
1103,657,Оптовая торговля,7,2018,14.428571


In [16]:
forecast['Направление продаж'] = forecast['Направление продаж'].apply(lambda x: 1 if (x == 'Розничная торговля') else 0)

In [17]:
forecast

,Код товара,Направление продаж,Month,Year,Скидка
0,1,1,7,2018,10.000000
1,2,0,7,2018,7.400000
2,4,1,8,2018,15.500000
3,5,0,9,2018,10.857143
4,6,0,7,2018,12.500000
...,...,...,...,...,...
1100,654,1,7,2018,15.000000
1101,654,1,9,2018,4.500000
1102,656,1,8,2018,12.000000
1103,657,0,7,2018,14.428571


In [18]:
training_df['Направление продаж'] = training_df['Направление продаж'].apply(lambda x: 1 if (x == 'Розничная торговля') else 0)

Поскольку нам придется добавлять недостающие сочетания в **forecast**, давайте посмотрим, сколько уникальных сочетаний в **forecast** и в **shipments** (training_df)

In [19]:
code_chan = []
counter = 0
for index, row in training_df.iterrows():
    code_chan.append((row['Код товара'], row['Направление продаж']))

In [20]:
code_chan = list(set(code_chan))
len(code_chan)

1292

In [21]:
code_chan_forecast = []
counter = 0
for index, row in forecast.iterrows():
    code_chan_forecast.append((row['Код товара'], row['Направление продаж']))

In [22]:
code_chan_forecast = list(set(code_chan_forecast))
len(code_chan_forecast)

829

Их оказалось очень много, будет проще создать отдельный датафрейм, который мы уже после заполним значениями из **forecast**

In [23]:
X_train = pd.DataFrame(columns = forecast.columns)
for combin in code_chan:
    jul = {'Код товара': combin[0], 'Направление продаж': combin[1], 'Month': 7, 'Year': 2018, 'Скидка': 0}
    X_train = X_train.append(jul, ignore_index = True)
    aug = {'Код товара': combin[0], 'Направление продаж': combin[1], 'Month': 8, 'Year': 2018, 'Скидка': 0}
    X_train = X_train.append(aug, ignore_index = True)
    sep = {'Код товара': combin[0], 'Направление продаж': combin[1], 'Month': 9, 'Year': 2018, 'Скидка': 0}
    X_train = X_train.append(sep, ignore_index = True)
X_train

,Код товара,Направление продаж,Month,Year,Скидка
0,521,0,7,2018,0
1,521,0,8,2018,0
2,521,0,9,2018,0
3,125,0,7,2018,0
4,125,0,8,2018,0
...,...,...,...,...,...
3871,46,0,8,2018,0
3872,46,0,9,2018,0
3873,175,1,7,2018,0
3874,175,1,8,2018,0


In [24]:
for index, row in X_train.iterrows():
    if (row['Код товара'], row['Направление продаж']) in code_chan_forecast:
        if row['Month'] in forecast.loc[(forecast['Код товара']==row['Код товара']) & (forecast['Направление продаж'] == row['Направление продаж'])]['Month'].to_list():
            row['Скидка'] = float(forecast.loc[(forecast['Код товара']==row['Код товара']) & (forecast['Направление продаж'] == row['Направление продаж']) & (forecast['Month'] == row['Month'])]['Скидка'])
X_train

,Код товара,Направление продаж,Month,Year,Скидка
0,521,0,7,2018,0
1,521,0,8,2018,11.5
2,521,0,9,2018,0
3,125,0,7,2018,8.0
4,125,0,8,2018,0
...,...,...,...,...,...
3871,46,0,8,2018,0
3872,46,0,9,2018,9.428571
3873,175,1,7,2018,0
3874,175,1,8,2018,0


In [25]:
month_dummies = pd.get_dummies(training_df.Month)
training_df = pd.concat([training_df, month_dummies], axis = 1)
training_df

,Код товара,Направление продаж,Month,Year,скидка,"Продажи, шт",1,2,3,4,5,6,7,8,9,10,11,12
0,1,1,1,2017,10.0,1052,1,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,2018,6.5,958,1,0,0,0,0,0,0,0,0,0,0,0
2,1,1,2,2017,8.0,1241,0,1,0,0,0,0,0,0,0,0,0,0
3,1,1,2,2018,13.0,868,0,1,0,0,0,0,0,0,0,0,0,0
4,1,1,3,2017,11.0,1630,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86520,657,0,6,2018,13.0,307,0,0,0,0,0,1,0,0,0,0,0,0
86521,657,0,7,2017,2.0,52,0,0,0,0,0,0,1,0,0,0,0,0
86522,657,0,10,2017,9.0,517,0,0,0,0,0,0,0,0,0,1,0,0
86523,657,0,11,2017,11.5,785,0,0,0,0,0,0,0,0,0,0,1,0


In [26]:
training_df.drop('Month', axis = 1, inplace = True)

In [27]:
training_df['Продажи'] = training_df['Продажи, шт']

In [28]:
training_df.drop('Продажи, шт', axis = 1, inplace = True)

In [29]:
training_df

,Код товара,Направление продаж,Year,скидка,1,2,3,4,5,6,7,8,9,10,11,12,Продажи
0,1,1,2017,10.0,1,0,0,0,0,0,0,0,0,0,0,0,1052
1,1,1,2018,6.5,1,0,0,0,0,0,0,0,0,0,0,0,958
2,1,1,2017,8.0,0,1,0,0,0,0,0,0,0,0,0,0,1241
3,1,1,2018,13.0,0,1,0,0,0,0,0,0,0,0,0,0,868
4,1,1,2017,11.0,0,0,1,0,0,0,0,0,0,0,0,0,1630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86520,657,0,2018,13.0,0,0,0,0,0,1,0,0,0,0,0,0,307
86521,657,0,2017,2.0,0,0,0,0,0,0,1,0,0,0,0,0,52
86522,657,0,2017,9.0,0,0,0,0,0,0,0,0,0,1,0,0,517
86523,657,0,2017,11.5,0,0,0,0,0,0,0,0,0,0,1,0,785


In [35]:
mon_table = pd.DataFrame(columns = training_df.columns[4:-1])

In [36]:
mon_table

,1,2,3,4,5,6,7,8,9,10,11,12


In [37]:
X_forpred = pd.concat([X_train, mon_table], axis = 1).fillna(0)

In [38]:
X_forpred

,Код товара,Направление продаж,Month,Year,Скидка,1,2,3,4,5,6,7,8,9,10,11,12
0,521,0,7,2018,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
1,521,0,8,2018,11.500000,0,0,0,0,0,0,0,0,0,0,0,0
2,521,0,9,2018,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
3,125,0,7,2018,8.000000,0,0,0,0,0,0,0,0,0,0,0,0
4,125,0,8,2018,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,46,0,8,2018,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
3872,46,0,9,2018,9.428571,0,0,0,0,0,0,0,0,0,0,0,0
3873,175,1,7,2018,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
3874,175,1,8,2018,0.000000,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
X_forpred.columns = ['Код товара','Направление продаж','Month','Year', 'Скидка', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
X_forpred.loc[(X_forpred.Month == 7), '7'] = 1
X_forpred.loc[(X_forpred.Month == 8), '8'] = 1
X_forpred.loc[(X_forpred.Month == 9), '9'] = 1
X_forpred.drop('Month', axis = 1, inplace = True)
X_forpred

,Код товара,Направление продаж,Year,Скидка,1,2,3,4,5,6,7,8,9,10,11,12
0,521,0,2018,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
1,521,0,2018,11.500000,0,0,0,0,0,0,0,1,0,0,0,0
2,521,0,2018,0.000000,0,0,0,0,0,0,0,0,1,0,0,0
3,125,0,2018,8.000000,0,0,0,0,0,0,1,0,0,0,0,0
4,125,0,2018,0.000000,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3871,46,0,2018,0.000000,0,0,0,0,0,0,0,1,0,0,0,0
3872,46,0,2018,9.428571,0,0,0,0,0,0,0,0,1,0,0,0
3873,175,1,2018,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
3874,175,1,2018,0.000000,0,0,0,0,0,0,0,1,0,0,0,0


In [40]:
X = training_df.drop('Продажи', axis = 1)
y = training_df['Продажи']

In [41]:
regressor = LinearRegression()  
regressor.fit(X, y)

LinearRegression()

In [42]:
y_pred = regressor.predict(X_forpred)